**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
from random import randint
import numpy as np
import io
import os
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

# That helps to compile a Keras model in my Macbook
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Knowing the actual state of the agent, the act function decides the next action to do. It takes into account the historical information with a high probability but sometimes explore new and random possibilities using epsilon.

Epsilon decides wheither we will explor or exploit: take a random action (with a probability of Epsilon) or choose the agent’s trusted action (with a probability of 1 -Epsilon). This strategy called greedy-epsilon strategy that means pick the current best option ("greedy") most of the time, but pick a random option with a small (epsilon) probability sometimes.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [28]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position represents all the possible cells where the agent can go (-1 otherwise). The array board contains the rewards corresponding to the movement towards these cells.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [29]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = randint(0, self.n_action - 1)
        return(a)
        
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        win   = 0
        lose  = 0
        end_game = False
        while not end_game:
            action = agent.act(state,train=False)
            state,reward,end_game = env.act(action)
            if reward >0:
                win += reward 
            if reward < 0:
                lose -= reward
        # Save as a mp4
        env.draw(prefix+str(e))
        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/12.0. Average score (-2.5)
Win/lose count 9.5/11.0. Average score (-2.0)
Win/lose count 9.0/7.0. Average score (-0.6666666666666666)
Win/lose count 8.5/17.0. Average score (-2.625)
Win/lose count 10.5/12.0. Average score (-2.4)
Win/lose count 14.0/10.0. Average score (-1.3333333333333333)
Win/lose count 3.0/9.0. Average score (-2.0)
Win/lose count 12.0/18.0. Average score (-2.5)
Win/lose count 12.5/16.0. Average score (-2.611111111111111)
Win/lose count 7.0/9.0. Average score (-2.55)
Win/lose count 10.0/18.0. Average score (-3.0454545454545454)
Win/lose count 12.5/10.0. Average score (-2.5833333333333335)
Win/lose count 11.5/9.0. Average score (-2.1923076923076925)
Win/lose count 9.0/23.0. Average score (-3.0357142857142856)
Win/lose count 11.5/17.0. Average score (-3.2)
Win/lose count 8.5/13.0. Average score (-3.28125)
Win/lose count 7.5/21.0. Average score (-3.8823529411764706)
Win/lose count 11.0/9.0. Average score (-3.5555555555555554)
Win/lose count 8.5/10.0. Av

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5.1__

We know that: $$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s_{0},a_{0}) + \sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
$$ Q^\pi(s,a)=E_{p^{\pi}}[ r(s,a) + \sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
$$ Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \gamma\sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
$$ Q^\pi(s,a)=  E_{p^{\pi}}[r(s,a) + \gamma\sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s^\prime,a_{1}=a^\prime] \> $$
Thus:

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$





__Question 5.2__

Since the optimal policy is:

$$ \pi^*(s) = \max_a Q^*(s,a) $$

Then

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

__Question 5.3__

We have here a minimisation problem :
\begin{equation*}
E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q^*(s',a',\theta)-Q^*(s,a,\theta)\Vert^{2} = 0 \text{ if Q = Q* and is strictly positive otherwise}
\end{equation*}

Thus minimizing the following objective is a viable option for training the DQN:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [43]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0] # delete the first element and keep only the latest moves

    def random_access(self):
        i = randint(0, len(self.memory)-1)
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [56]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) #simple one hidden layer network (see performance of other methods below)
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/040 | Loss 0.0311 | Win/lose count 2.0/1.0 (1.0)
Epoch 001/040 | Loss 0.1182 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/040 | Loss 0.0374 | Win/lose count 0.5/4.0 (-3.5)
Epoch 003/040 | Loss 0.2876 | Win/lose count 1.0/3.0 (-2.0)
Epoch 004/040 | Loss 0.0212 | Win/lose count 1.0/2.0 (-1.0)
Epoch 005/040 | Loss 0.0254 | Win/lose count 0.5/2.0 (-1.5)
Epoch 006/040 | Loss 0.0450 | Win/lose count 1.0/5.0 (-4.0)
Epoch 007/040 | Loss 0.0494 | Win/lose count 1.5/4.0 (-2.5)
Epoch 008/040 | Loss 0.0808 | Win/lose count 0.5/0 (0.5)
Epoch 009/040 | Loss 0.0210 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/040 | Loss 0.0388 | Win/lose count 1.5/4.0 (-2.5)
Epoch 011/040 | Loss 0.0222 | Win/lose count 1.5/3.0 (-1.5)
Epoch 012/040 | Loss 0.0104 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/040 | Loss 0.0345 | Win/lose count 2.5/3.0 (-0.5)
Epoch 014/040 | Loss 0.0137 | Win/lose count 4.0/5.0 (-1.0)
Epoch 015/040 | Loss 0.0114 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/040 | Loss 0.0170 | Win/lose count 1.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [58]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Conv2D(60,(2,2),input_shape=(5,5,self.n_state),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4)) # since we have 4 steps
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/040 | Loss 0.0080 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/040 | Loss 0.0120 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/040 | Loss 0.0004 | Win/lose count 1.0/3.0 (-2.0)
Epoch 003/040 | Loss 0.0233 | Win/lose count 0.5/8.0 (-7.5)
Epoch 004/040 | Loss 0.0047 | Win/lose count 2.5/1.0 (1.5)
Epoch 005/040 | Loss 0.0028 | Win/lose count 1.0/2.0 (-1.0)
Epoch 006/040 | Loss 0.0141 | Win/lose count 3.5/0 (3.5)
Epoch 007/040 | Loss 0.0024 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/040 | Loss 0.0156 | Win/lose count 2.0/3.0 (-1.0)
Epoch 009/040 | Loss 0.0064 | Win/lose count 1.0/0 (1.0)
Epoch 010/040 | Loss 0.0047 | Win/lose count 1.0/0 (1.0)
Epoch 011/040 | Loss 0.0026 | Win/lose count 1.0/2.0 (-1.0)
Epoch 012/040 | Loss 0.0123 | Win/lose count 0.5/1.0 (-0.5)
Epoch 013/040 | Loss 0.0023 | Win/lose count 2.5/3.0 (-0.5)
Epoch 014/040 | Loss 0.0003 | Win/lose count 0.5/0 (0.5)
Epoch 015/040 | Loss 0.0023 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/040 | Loss 0.0026 | Win/lose count 1.0/4.0 (-

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [60]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 0.5/1.0. Average score (-0.25)
Win/lose count 1.0/0. Average score (0.16666666666666666)
Win/lose count 0/0. Average score (0.125)
Win/lose count 2.0/1.0. Average score (0.3)
Win/lose count 0.5/1.0. Average score (0.16666666666666666)
Win/lose count 0.5/0. Average score (0.21428571428571427)
Win/lose count 0.5/0. Average score (0.25)
Win/lose count 0/1.0. Average score (0.1111111111111111)
Win/lose count 0.5/1.0. Average score (0.05)
Win/lose count 0.5/1.0. Average score (0.0)
Win/lose count 0.5/2.0. Average score (-0.125)
Win/lose count 1.0/0. Average score (-0.038461538461538464)
Win/lose count 2.0/2.0. Average score (-0.03571428571428571)
Win/lose count 2.5/2.0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.5/1.0. Average score (0.027777777777777776)
Win/lose count 0/1.0. Average score (-0.02631578947368421)
Win/lose count 0/1.0. Average s

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

In [61]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.0/1.0. Average score (5.0)
Win/lose count 7.0/1.0. Average score (5.5)
Win/lose count 0.5/1.0. Average score (3.5)
Win/lose count 0.5/0. Average score (2.75)
Win/lose count 7.5/0. Average score (3.7)
Win/lose count 5.0/1.0. Average score (3.75)
Win/lose count 6.0/1.0. Average score (3.9285714285714284)
Win/lose count 3.0/0. Average score (3.8125)
Win/lose count 6.5/0. Average score (4.111111111111111)
Win/lose count 0.5/0. Average score (3.75)
Win/lose count 4.5/0. Average score (3.8181818181818183)
Win/lose count 3.0/0. Average score (3.75)
Win/lose count 12.5/1.0. Average score (4.346153846153846)
Win/lose count 12.5/3.0. Average score (4.714285714285714)
Win/lose count 8.0/3.0. Average score (4.733333333333333)
Win/lose count 2.5/1.0. Average score (4.53125)
Win/lose count 1.5/0. Average score (4.352941176470588)
Win/lose count 1.0/0. Average score (4.166666666666667)
Win/lose count 4.5/2.0. Average score (4.078947368421052)
Win/lose count 6.0/2.0. A

According to several results that i have gotten, i can say that the more the temperature is high, the more the agents move. In general, CNN performs much better comparing to FC. But, they both suffer from one issue. They dont explore many cells, that what causes their stagnant positions.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [52]:
def train_explore(agent,env,epoch,prefix=''):
    
    decay_epsilon=0.3
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        # we decrease epsilon by decay_epsilon rate 
        agent.set_epsilon(agent.epsilon*(1-decay_epsilon))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state               = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size        = grid_size+4
        self.grid_size   = grid_size
        self.max_time    = max_time
        self.temperature = temperature

        #board on which one plays
        self.board          = np.zeros((grid_size,grid_size))
        self.position       = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # tells if the cell has been visited or not yet
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t     = 0
        self.scale =16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward                     = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over                  = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [59]:
# Training
env   = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/040 | Loss 0.0148 | Win/lose count 7.5/25.300000000000082 (-17.800000000000082)
Epoch 001/040 | Loss 0.0061 | Win/lose count 7.0/17.39999999999998 (-10.39999999999998)
Epoch 002/040 | Loss 0.0094 | Win/lose count 6.0/20.300000000000026 (-14.300000000000026)
Epoch 003/040 | Loss 0.0062 | Win/lose count 4.0/18.599999999999994 (-14.599999999999994)
Epoch 004/040 | Loss 0.0085 | Win/lose count 11.5/15.899999999999963 (-4.399999999999963)
Epoch 005/040 | Loss 0.0153 | Win/lose count 8.0/17.199999999999978 (-9.199999999999978)
Epoch 006/040 | Loss 0.0036 | Win/lose count 7.0/17.099999999999973 (-10.099999999999973)
Epoch 007/040 | Loss 0.0078 | Win/lose count 2.5/18.599999999999994 (-16.099999999999994)
Epoch 008/040 | Loss 0.0031 | Win/lose count 4.5/18.29999999999999 (-13.79999999999999)
Epoch 009/040 | Loss 0.0100 | Win/lose count 7.0/17.399999999999977 (-10.399999999999977)
Epoch 010/040 | Loss 0.0045 | Win/lose count 10.5/18.199999999999992 (-7.699999999999992)
Epoch 011/040 |

In [19]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 24.5/0. Average score (24.5)
Win/lose count 19.5/2.0. Average score (21.0)
Win/lose count 21.5/1.0. Average score (20.833333333333332)
Win/lose count 24.0/0. Average score (21.625)
Win/lose count 17.0/0. Average score (20.7)
Win/lose count 23.5/5.0. Average score (20.333333333333332)
Win/lose count 12.5/0. Average score (19.214285714285715)
Win/lose count 15.0/0. Average score (18.6875)
Win/lose count 8.0/0. Average score (17.5)
Win/lose count 12.5/0. Average score (17.0)
Win/lose count 21.5/3.0. Average score (17.136363636363637)
Win/lose count 19.5/2.0. Average score (17.166666666666668)
Win/lose count 17.5/1.0. Average score (17.115384615384617)
Win/lose count 22.5/0. Average score (17.5)
Win/lose count 21.0/1.0. Average score (17.666666666666668)
Win/lose count 26.5/2.0. Average score (18.09375)
Win/lose count 20.5/1.0. Average score (18.176470588235293)
Win/lose count 26.5/3.0. Average score (18.47222222222222)
Win/lose count 29.0/1.0. Average score (18.973684210526

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [66]:
def train_mimick(expert , agent ,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action , train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = expert.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        expert.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [70]:
# Training
env_explor =  EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
expert     = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_mimick(expert , agent, env_explor, epochs_train, prefix='cnn_train_mimick')
HTML(display_videos('cnn_train_mimick10.mp4'))

Epoch 000/040 | Loss 0.0058 | Win/lose count 2.0/19.400000000000006 (-17.400000000000006)
Epoch 001/040 | Loss 0.0134 | Win/lose count 1.5/22.200000000000053 (-20.700000000000053)
Epoch 002/040 | Loss 0.0176 | Win/lose count 0.5/20.500000000000025 (-20.000000000000025)
Epoch 003/040 | Loss 0.0155 | Win/lose count 2.0/21.20000000000004 (-19.20000000000004)
Epoch 004/040 | Loss 0.0082 | Win/lose count 0/21.500000000000043 (-21.500000000000043)
Epoch 005/040 | Loss 0.0060 | Win/lose count 1.0/20.500000000000025 (-19.500000000000025)
Epoch 006/040 | Loss 0.0078 | Win/lose count 1.5/26.600000000000094 (-25.100000000000094)
Epoch 007/040 | Loss 0.0061 | Win/lose count 1.0/21.70000000000003 (-20.70000000000003)
Epoch 008/040 | Loss 0.0159 | Win/lose count 0.5/21.300000000000036 (-20.800000000000036)
Epoch 009/040 | Loss 0.0059 | Win/lose count 1.5/20.80000000000002 (-19.30000000000002)
Epoch 010/040 | Loss 0.0089 | Win/lose count 1.5/19.0 (-17.5)
Epoch 011/040 | Loss 0.0140 | Win/lose count 0

In [71]:
# Testing
test(expert,env_explor,epochs_test,prefix='cnn_test_mimick')
HTML(display_videos('cnn_test_mimick10.mp4'))

Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Final score: 0.0


We conclude that considering this problem as a supervised problem is noit a good approach to get improved result. Indeed, the agent (expert) that learns from the first one, cannot learn a lot of information, thus we get poor performances.

***